for xml

In [ ]:
import os
import shutil
from pathlib import Path
from collections import defaultdict

def verify_and_fix_dataset(output_path):
    """
    Verify the converted dataset and fix any issues
    """
    print("🔍 VERIFYING CONVERTED DATASET")
    print("=" * 50)
    
    output_dir = Path(output_path)
    splits = ['train', 'valid', 'test']
    
    total_images = 0
    class_distribution = {}
    
    for split in splits:
        split_dir = output_dir / split
        if not split_dir.exists():
            print(f"❌ Split directory missing: {split}")
            continue
            
        print(f"\n📁 Checking {split.upper()} split:")
        print("-" * 30)
        
        split_total = 0
        split_classes = {}
        
        for class_dir in split_dir.iterdir():
            if class_dir.is_dir():
                class_name = class_dir.name
                images = list(class_dir.glob('*'))
                image_count = len([img for img in images if img.is_file()])
                
                split_classes[class_name] = image_count
                split_total += image_count
                
                print(f"  📂 {class_name}: {image_count} images")
                
                # Show a few sample filenames
                sample_images = [img.name for img in images if img.is_file()][:3]
                if sample_images:
                    print(f"     Sample files: {sample_images}")
        
        class_distribution[split] = split_classes
        total_images += split_total
        print(f"  Total in {split}: {split_total} images")
    
    print(f"\n📊 SUMMARY:")
    print(f"Total images in converted dataset: {total_images}")
    print(f"Expected: 248 images")
    
    if total_images != 248:
        print(f"⚠️  Missing {248 - total_images} images!")
    
    # Check for missing classes
    expected_classes = ['Domestic_Trash_Garbage', 'dw', 'trash']
    all_found_classes = set()
    
    for split_data in class_distribution.values():
        all_found_classes.update(split_data.keys())
    
    missing_classes = set(expected_classes) - all_found_classes
    if missing_classes:
        print(f"❌ Missing classes entirely: {missing_classes}")
    
    return class_distribution

def fix_missing_images(kaggle_images_path, kaggle_annotations_path, output_path):
    """
    Re-process images that didn't get copied correctly
    """
    print(f"\n🔧 FIXING MISSING IMAGES")
    print("=" * 50)
    
    import xml.etree.ElementTree as ET
    import random
    
    # Re-read annotations
    annotations_path = Path(kaggle_annotations_path)
    images_path = Path(kaggle_images_path)
    output_dir = Path(output_path)
    
    problem_images = []
    
    # Look for "Domestic Trash_Garbage" class specifically
    xml_files = list(annotations_path.glob('*.xml'))
    
    for xml_file in xml_files:
        try:
            tree = ET.parse(xml_file)
            root = tree.getroot()
            
            # Get filename
            filename_elem = root.find('filename')
            if filename_elem is not None:
                filename = filename_elem.text
                filename_stem = Path(filename).stem
            else:
                filename_stem = xml_file.stem
                filename = f"{filename_stem}.jpg"  # assume jpg
            
            # Get classes
            objects = root.findall('object')
            classes_in_image = []
            
            for obj in objects:
                name_elem = obj.find('name')
                if name_elem is not None:
                    class_name = name_elem.text.strip()
                    if class_name == 'Domestic Trash\tGarbage':  # Original name with tab
                        classes_in_image.append('Domestic_Trash_Garbage')  # Clean name
                    else:
                        classes_in_image.append(class_name)
            
            # If this image has the rare class, let's manually place it
            if 'Domestic_Trash_Garbage' in classes_in_image:
                print(f"🔍 Found Domestic_Trash_Garbage image: {filename}")
                
                # Find the actual image file
                image_found = False
                possible_extensions = ['.jpg', '.jpeg', '.png', '.JPG', '.JPEG', '.PNG']
                
                for ext in possible_extensions:
                    img_path = images_path / f"{filename_stem}{ext}"
                    if img_path.exists():
                        # Copy to train folder
                        dest_path = output_dir / 'train' / 'Domestic_Trash_Garbage' / img_path.name
                        
                        try:
                            shutil.copy2(img_path, dest_path)
                            print(f"✅ Manually copied: {img_path.name} -> train/Domestic_Trash_Garbage/")
                            image_found = True
                            break
                        except Exception as e:
                            print(f"❌ Error copying {img_path.name}: {e}")
                
                if not image_found:
                    print(f"❌ Could not find image file for: {filename}")
                    problem_images.append(filename)
                    
        except Exception as e:
            print(f"❌ Error processing {xml_file.name}: {e}")
    
    return problem_images

def create_stratified_split(kaggle_images_path, kaggle_annotations_path, output_path):
    """
    Create a better split that ensures all classes appear in training
    """
    print(f"\n🎯 CREATING STRATIFIED SPLIT")
    print("=" * 50)
    
    import xml.etree.ElementTree as ET
    import random
    from collections import defaultdict
    
    # Group images by class
    class_to_images = defaultdict(list)
    
    annotations_path = Path(kaggle_annotations_path)
    images_path = Path(kaggle_images_path)
    
    xml_files = list(annotations_path.glob('*.xml'))
    
    for xml_file in xml_files:
        try:
            tree = ET.parse(xml_file)
            root = tree.getroot()
            
            # Get filename
            filename_elem = root.find('filename')
            if filename_elem is not None:
                filename = filename_elem.text
                filename_stem = Path(filename).stem
            else:
                filename_stem = xml_file.stem
            
            # Find actual image file
            image_path = None
            possible_extensions = ['.jpg', '.jpeg', '.png', '.JPG', '.JPEG', '.PNG']
            
            for ext in possible_extensions:
                img_path = images_path / f"{filename_stem}{ext}"
                if img_path.exists():
                    image_path = img_path
                    break
            
            if not image_path:
                continue
            
            # Get classes
            objects = root.findall('object')
            for obj in objects:
                name_elem = obj.find('name')
                if name_elem is not None:
                    class_name = name_elem.text.strip()
                    
                    # Clean class name
                    if class_name == 'Domestic Trash\tGarbage':
                        clean_class = 'Domestic_Trash_Garbage'
                    else:
                        clean_class = class_name
                    
                    class_to_images[clean_class].append(image_path)
                    break  # Only use first class for simplicity
                    
        except Exception as e:
            print(f"Error processing {xml_file.name}: {e}")
    
    print(f"📊 Images per class:")
    for class_name, images in class_to_images.items():
        print(f"  {class_name}: {len(images)} images")
    
    # Ensure each class has at least 1 image in training
    output_dir = Path(output_path)
    
    # Clear existing structure
    if output_dir.exists():
        shutil.rmtree(output_dir)
    
    splits = ['train', 'valid', 'test']
    for split in splits:
        for class_name in class_to_images.keys():
            (output_dir / split / class_name).mkdir(parents=True, exist_ok=True)
    
    # Stratified split
    for class_name, images in class_to_images.items():
        random.shuffle(images)
        
        total = len(images)
        if total == 1:
            # Put single image in training
            train_images = images
            valid_images = []
            test_images = []
        elif total <= 3:
            # Put all in training for very small classes
            train_images = images
            valid_images = []
            test_images = []
        else:
            # Normal split but ensure at least 1 in training
            train_end = max(1, int(total * 0.7))
            valid_end = train_end + int(total * 0.2)
            
            train_images = images[:train_end]
            valid_images = images[train_end:valid_end]
            test_images = images[valid_end:]
        
        # Copy images
        for img_path in train_images:
            dest = output_dir / 'train' / class_name / img_path.name
            shutil.copy2(img_path, dest)
        
        for img_path in valid_images:
            dest = output_dir / 'valid' / class_name / img_path.name
            shutil.copy2(img_path, dest)
        
        for img_path in test_images:
            dest = output_dir / 'test' / class_name / img_path.name
            shutil.copy2(img_path, dest)
        
        print(f"✅ {class_name}: {len(train_images)} train, {len(valid_images)} valid, {len(test_images)} test")

# Example usage
if __name__ == "__main__":
    # Update these paths
    kaggle_annotations_path = "./concrete_damage_detect/annot"
    kaggle_images_path = "./concrete_damage_detect/img"
    output_path = "./concrete-damage"
    
    print("STEP 1: Verify current dataset")
    class_dist = verify_and_fix_dataset(output_path)
    
    print("\nSTEP 2: Fix missing images")
    problem_images = fix_missing_images(kaggle_images_path, kaggle_annotations_path, output_path)
    
    print("\nSTEP 3: Re-verify dataset")
    verify_and_fix_dataset(output_path)
    
    # If still problems, recreate with stratified split
    user_input = input("\nDo you want to recreate the dataset with stratified split? (y/n): ")
    if user_input.lower() == 'y':
        print("\nSTEP 4: Creating stratified split")
        create_stratified_split(kaggle_images_path, kaggle_annotations_path, output_path)
        print("\nSTEP 5: Final verification")
        verify_and_fix_dataset(output_path)

for json

In [2]:
import os
import json
import shutil
from pathlib import Path
from collections import defaultdict
import random

def verify_and_fix_dataset(output_path):
    """
    Verify the converted dataset and fix any issues
    """
    print("🔍 VERIFYING CONVERTED DATASET")
    print("=" * 50)
    
    output_dir = Path(output_path)
    splits = ['train', 'valid', 'test']
    
    total_images = 0
    class_distribution = {}
    
    for split in splits:
        split_dir = output_dir / split
        if not split_dir.exists():
            print(f"❌ Split directory missing: {split}")
            continue
            
        print(f"\n📁 Checking {split.upper()} split:")
        print("-" * 30)
        
        split_total = 0
        split_classes = {}
        
        for class_dir in split_dir.iterdir():
            if class_dir.is_dir():
                class_name = class_dir.name
                images = list(class_dir.glob('*'))
                image_count = len([img for img in images if img.is_file()])
                
                split_classes[class_name] = image_count
                split_total += image_count
                
                print(f"  📂 {class_name}: {image_count} images")
                
                # Show a few sample filenames
                sample_images = [img.name for img in images if img.is_file()][:3]
                if sample_images:
                    print(f"     Sample files: {sample_images}")
        
        class_distribution[split] = split_classes
        total_images += split_total
        print(f"  Total in {split}: {split_total} images")
    
    print(f"\n📊 SUMMARY:")
    print(f"Total images in converted dataset: {total_images}")
    
    # Get unique classes across all splits
    all_found_classes = set()
    for split_data in class_distribution.values():
        all_found_classes.update(split_data.keys())
    
    print(f"Found classes: {sorted(all_found_classes)}")
    
    return class_distribution

def load_json_annotations(json_path):
    """
    Load annotations from JSON file and return structured data
    Supports both COCO format and custom formats
    """
    print(f"📖 Loading JSON annotations from: {json_path}")
    
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    image_annotations = {}
    
    # Check if it's COCO format
    if 'images' in data and 'annotations' in data and 'categories' in data:
        print("🔍 Detected COCO format")
        
        # Create category mapping
        categories = {cat['id']: cat['name'] for cat in data['categories']}
        
        # Create image mapping
        images = {img['id']: img for img in data['images']}
        
        # Group annotations by image
        for ann in data['annotations']:
            image_id = ann['image_id']
            if image_id not in image_annotations:
                image_annotations[image_id] = {
                    'filename': images[image_id]['file_name'],
                    'classes': []
                }
            
            category_name = categories[ann['category_id']]
            if category_name not in image_annotations[image_id]['classes']:
                image_annotations[image_id]['classes'].append(category_name)
    
    # Check if it's a simple format with direct image-class mapping
    elif isinstance(data, dict):
        print("🔍 Detected custom dict format")
        
        for filename, info in data.items():
            if isinstance(info, dict) and 'class' in info:
                class_name = info['class']
            elif isinstance(info, dict) and 'classes' in info:
                class_name = info['classes'] if isinstance(info['classes'], str) else info['classes'][0]
            elif isinstance(info, str):
                class_name = info
            else:
                continue
                
            image_annotations[filename] = {
                'filename': filename,
                'classes': [class_name] if isinstance(class_name, str) else class_name
            }
    
    # Check if it's a list format
    elif isinstance(data, list):
        print("🔍 Detected list format")
        
        for item in data:
            if 'filename' in item and ('class' in item or 'classes' in item):
                filename = item['filename']
                classes = item.get('classes', [item.get('class', '')])
                if isinstance(classes, str):
                    classes = [classes]
                
                image_annotations[filename] = {
                    'filename': filename,
                    'classes': classes
                }
    
    else:
        raise ValueError("Unsupported JSON format. Please check your annotation file structure.")
    
    print(f"✅ Loaded annotations for {len(image_annotations)} images")
    
    return image_annotations

def clean_class_name(class_name):
    """
    Clean class names by replacing problematic characters
    """
    # Replace tabs, spaces, and other problematic characters
    cleaned = class_name.replace('\t', '_').replace(' ', '_').replace('/', '_').replace('\\', '_')
    # Remove multiple underscores
    while '__' in cleaned:
        cleaned = cleaned.replace('__', '_')
    return cleaned.strip('_')

def create_stratified_split(images_path, json_annotations_path, output_path, train_ratio=0.7, valid_ratio=0.2, test_ratio=0.1):
    """
    Create a stratified split from JSON annotations
    """
    print(f"\n🎯 CREATING STRATIFIED SPLIT")
    print("=" * 50)
    
    # Load annotations
    image_annotations = load_json_annotations(json_annotations_path)
    
    # Group images by class
    class_to_images = defaultdict(list)
    images_dir = Path(images_path)
    
    # Find image files and match with annotations
    for annotation_key, annotation_data in image_annotations.items():
        filename = annotation_data['filename']
        classes = annotation_data['classes']
        
        # Find the actual image file
        image_path = None
        filename_stem = Path(filename).stem
        
        # Try exact match first
        if (images_dir / filename).exists():
            image_path = images_dir / filename
        else:
            # Try different extensions
            possible_extensions = ['.jpg', '.jpeg', '.png', '.JPG', '.JPEG', '.PNG', '.bmp', '.BMP']
            
            for ext in possible_extensions:
                img_path = images_dir / f"{filename_stem}{ext}"
                if img_path.exists():
                    image_path = img_path
                    break
        
        if not image_path:
            print(f"⚠️  Image not found: {filename}")
            continue
        
        # Use the first class for stratification (or modify as needed)
        if classes:
            primary_class = clean_class_name(classes[0])
            class_to_images[primary_class].append((image_path, classes))
    
    print(f"📊 Images per class:")
    total_images = 0
    for class_name, images in class_to_images.items():
        print(f"  {class_name}: {len(images)} images")
        total_images += len(images)
    
    print(f"Total images found: {total_images}")
    
    # Create output directory structure
    output_dir = Path(output_path)
    
    # Clear existing structure
    if output_dir.exists():
        shutil.rmtree(output_dir)
    
    splits = ['train', 'valid', 'test']
    for split in splits:
        for class_name in class_to_images.keys():
            (output_dir / split / class_name).mkdir(parents=True, exist_ok=True)
    
    # Perform stratified split
    print(f"\n📁 Creating splits (train: {train_ratio}, valid: {valid_ratio}, test: {test_ratio})")
    
    for class_name, image_data_list in class_to_images.items():
        random.shuffle(image_data_list)
        
        total = len(image_data_list)
        
        if total == 1:
            # Put single image in training
            train_data = image_data_list
            valid_data = []
            test_data = []
        elif total <= 3:
            # Put all in training for very small classes
            train_data = image_data_list
            valid_data = []
            test_data = []
        else:
            # Normal split but ensure at least 1 in training
            train_end = max(1, int(total * train_ratio))
            valid_end = train_end + int(total * valid_ratio)
            
            train_data = image_data_list[:train_end]
            valid_data = image_data_list[train_end:valid_end]
            test_data = image_data_list[valid_end:]
        
        # Copy images to respective folders
        for img_path, classes in train_data:
            dest = output_dir / 'train' / class_name / img_path.name
            shutil.copy2(img_path, dest)
        
        for img_path, classes in valid_data:
            dest = output_dir / 'valid' / class_name / img_path.name
            shutil.copy2(img_path, dest)
        
        for img_path, classes in test_data:
            dest = output_dir / 'test' / class_name / img_path.name
            shutil.copy2(img_path, dest)
        
        print(f"✅ {class_name}: {len(train_data)} train, {len(valid_data)} valid, {len(test_data)} test")

def create_class_mapping_file(output_path):
    """
    Create a class mapping file for reference
    """
    output_dir = Path(output_path)
    
    # Get all classes from train directory
    train_dir = output_dir / 'train'
    if not train_dir.exists():
        return
    
    classes = sorted([d.name for d in train_dir.iterdir() if d.is_dir()])
    
    # Create class mapping
    class_mapping = {i: class_name for i, class_name in enumerate(classes)}
    
    # Save to file
    mapping_file = output_dir / 'class_mapping.json'
    with open(mapping_file, 'w', encoding='utf-8') as f:
        json.dump(class_mapping, f, indent=2, ensure_ascii=False)
    
    print(f"📄 Class mapping saved to: {mapping_file}")
    print(f"Classes: {classes}")

def analyze_json_structure(json_path):
    """
    Analyze the structure of the JSON file to help understand the format
    """
    print(f"🔍 ANALYZING JSON STRUCTURE")
    print("=" * 50)
    
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    print(f"JSON Type: {type(data)}")
    
    if isinstance(data, dict):
        print(f"Top-level keys: {list(data.keys())}")
        
        # Show sample of first few items
        sample_keys = list(data.keys())[:3]
        for key in sample_keys:
            print(f"Sample '{key}': {data[key]}")
            
    elif isinstance(data, list):
        print(f"List length: {len(data)}")
        if data:
            print(f"First item keys: {list(data[0].keys()) if isinstance(data[0], dict) else 'Not a dict'}")
            print(f"Sample item: {data[0]}")
    
    print("\n" + "=" * 50)

# Example usage
if __name__ == "__main__":
    # Update these paths to match your setup
    json_annotations_path = "./Branch-1/_annotations.coco.json"  # Your JSON annotation file
    images_path = "./Branch-1/train"  # Directory containing your images
    output_path = "./branches-road"  # Output directory for train/valid/test split
    
    print("STEP 1: Analyze JSON structure")
    analyze_json_structure(json_annotations_path)
    
    print("\nSTEP 2: Create stratified split")
    create_stratified_split(
        images_path=images_path,
        json_annotations_path=json_annotations_path,
        output_path=output_path,
        train_ratio=0.7,
        valid_ratio=0.2,
        test_ratio=0.1
    )
    
    print("\nSTEP 3: Create class mapping file")
    create_class_mapping_file(output_path)
    
    print("\nSTEP 4: Verify final dataset")
    verify_and_fix_dataset(output_path)

STEP 1: Analyze JSON structure
🔍 ANALYZING JSON STRUCTURE
JSON Type: <class 'dict'>
Top-level keys: ['info', 'licenses', 'categories', 'images', 'annotations']
Sample 'info': {'year': '2025', 'version': '1', 'description': 'Exported from roboflow.com', 'contributor': '', 'url': 'https://public.roboflow.com/object-detection/undefined', 'date_created': '2025-08-09T10:49:50+00:00'}
Sample 'licenses': [{'id': 1, 'url': 'https://creativecommons.org/licenses/by/4.0/', 'name': 'CC BY 4.0'}]
Sample 'categories': [{'id': 0, 'name': 'Branch', 'supercategory': 'none'}, {'id': 1, 'name': '0', 'supercategory': 'Branch'}]


STEP 2: Create stratified split

🎯 CREATING STRATIFIED SPLIT
📖 Loading JSON annotations from: ./Branch-1/_annotations.coco.json
🔍 Detected COCO format
✅ Loaded annotations for 820 images
📊 Images per class:
  0: 820 images
Total images found: 820

📁 Creating splits (train: 0.7, valid: 0.2, test: 0.1)
✅ 0: 574 train, 164 valid, 82 test

STEP 3: Create class mapping file
📄 Class ma

In [3]:
import os
import shutil
import random
from pathlib import Path

# Paths
dataset_path = Path("parking-illegal-final-1/train")
output_path = Path("parking-illegal-final-1-split")

# Split ratios
train_ratio = 0.7
valid_ratio = 0.2
test_ratio = 0.1

# Make sure output directories exist
for split in ["train", "valid", "test"]:
    for class_dir in os.listdir(dataset_path):
        (output_path / split / class_dir).mkdir(parents=True, exist_ok=True)

# Process each class folder
for class_dir in os.listdir(dataset_path):
    class_path = dataset_path / class_dir
    if not class_path.is_dir():
        continue

    images = list(class_path.glob("*"))
    random.shuffle(images)

    total = len(images)
    train_end = int(total * train_ratio)
    valid_end = train_end + int(total * valid_ratio)

    # Move files
    for img in images[:train_end]:
        shutil.copy(img, output_path / "train" / class_dir / img.name)
    for img in images[train_end:valid_end]:
        shutil.copy(img, output_path / "valid" / class_dir / img.name)
    for img in images[valid_end:]:
        shutil.copy(img, output_path / "test" / class_dir / img.name)

print("✅ Dataset split completed!")


✅ Dataset split completed!


In [10]:
import os

root_dir = r"C:\Users\TIME IT COMPUTERS\Desktop\datathon vision"
max_depth = 3
max_files_in_last_level = 2

for dirpath, dirnames, filenames in os.walk(root_dir):
    level = dirpath.replace(root_dir, "").count(os.sep)
    
    if level > max_depth:
        dirnames.clear()
        continue

    indent = " " * 4 * level
    print(f"{indent}{os.path.basename(dirpath)}/")

    subindent = " " * 4 * (level + 1)
    
    # If at last allowed level, limit number of files printed
    if level == max_depth:
        files_to_show = filenames[:max_files_in_last_level]
    else:
        files_to_show = filenames

    for f in files_to_show:
        print(f"{subindent}{f}")


datathon vision/
    annotation.ipynb
    dataset_download.ipynb
    split.ipynb
    txt.md
    branches-road/
        class_mapping.json
        test/
            branches/
                16_jpg.rf.f0d20415c8ad6c1e0fc69696830c611b.jpg
                48_jpg.rf.913b49777719db995b699fa9baa581d6.jpg
        train/
            branches/
                10_jpg.rf.b1d9140818d2d0b291f2f2831a5fd0b4.jpg
                10_jpg.rf.bf3f34e5a9367cff40fe2842a4cdb046.jpg
        valid/
            branches/
                13_jpg.rf.26bcc6be75f813f76ea03db294c8667e.jpg
                14_jpg.rf.efbe023b3e2b6b1b2d9c06f18f57cc17.jpg
    concrete-damage/
        test/
            amage/
            crack/
                367623.png
                368203.png
            damage/
                371103.png
                372031.png
        train/
            amage/
                y1.4792.png
            crack/
                367634.png
                367801.png
            damage/
                36

        valid/
            amage/
            crack/
                368310.png
                368830.png
            damage/
                368200.png
                368201.png
    damaged-signs-Hind-2/
        README.dataset.txt
        README.roboflow.txt
        test/
            _tokenization.txt
            Healthy/
                IMG_8070_jpg.rf.e0a9e5acaa39c419fbfc5bbd346ed3b2.jpg
                IMG_8073_jpg.rf.2c18af42211d0222ae00af057c496dd8.jpg
            Healthy_multiple/
                IMG_8084_jpg.rf.f45f201eac8b497abfa2d2cad13c2054.jpg
                IMG_8114_jpg.rf.c79f5c72cff13821c2b0f2709fddca94.jpg
        train/
            _tokenization.txt
            Damage/
                image0-20-_jpeg_jpg.rf.63866fb2d3b189699fa33b20fcc1db2f.jpg
                image0-20-_jpeg_jpg.rf.7b4cf51fa1b22b6316d29e43711f1cdc.jpg
            Damage_multiple/
                IMG_8720_jpg.rf.f28725991af0a6e61969358d67c30e4d.jpg
                IMG_8720_jpg.rf.fcca0a42ce34fe246281

In [3]:
import os

def print_tree_limited(startpath, prefix=""):
    entries = sorted(os.listdir(startpath))
    
    # Separate directories and files
    dirs = [e for e in entries if os.path.isdir(os.path.join(startpath, e))]
    files = [e for e in entries if os.path.isfile(os.path.join(startpath, e))]
    
    # Limit files to first 3
    limited_files = files[:3]
    
    # Combine dirs + limited files
    combined = dirs + limited_files
    
    for i, entry in enumerate(combined):
        path = os.path.join(startpath, entry)
        connector = "├── " if i < len(combined) - 1 else "└── "
        print(prefix + connector + entry)
        if os.path.isdir(path):
            extension = "│   " if i < len(combined) - 1 else "    "
            print_tree_limited(path, prefix + extension)

# Usage:
print_tree_limited('./')


├── Accident-1
│   ├── test
│   │   ├── Accident Detection - v2 Reannotated-2024-02-20 6-34am
│   │   │   ├── 1-99_jpg.rf.3f0bc8ae96f3c47081bb5d03cf6a7a50.jpg
│   │   │   ├── 2accidentFrame826_jpg.rf.7bf43da4e5c679ff43be076e17d090c8.jpg
│   │   │   └── 905_jpeg_jpg.rf.2ed1fd5cbe693d33cf3a88af7a872895.jpg
│   │   ├── Accident Detection - v2 Reannotated-2024-02-20 6-34am_multiple
│   │   │   ├── Severity_Detect_V9_jpg.rf.b5af253e33b391d926abedb476de645f.jpg
│   │   │   ├── test11_32_jpg.rf.486a9c0b80fc1d71686a4313a7aea58f.jpg
│   │   │   └── vlcsnap-2020-01-16-01h35m23s572_png_jpg.rf.4fa9e66b6eba9c8f8ef19ea7d489d832.jpg
│   │   └── _tokenization.txt
│   ├── train
│   │   ├── Accident Detection - v2 Reannotated-2024-02-20 6-34am
│   │   │   ├── 1-44_jpg.rf.ac072eb8ee3b73f7b0cf45b5013dd9be.jpg
│   │   │   ├── 1001_jpeg_jpg.rf.b2cd23cafdd6845de403c2b6d757dd00.jpg
│   │   │   └── 1075_jpeg_jpg.rf.0b4cf8e68a8f8d16044b35c47e8672f4.jpg
│   │   ├── Accident Detection - v2 Reannotated-2024-02-20 